# Get the Data

In [1]:
import pandas as pd

In [2]:
## tell my computer where the data is

DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'

# Train/Test Split

In [3]:
## split data into training and testing
from sklearn.model_selection import train_test_split

train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

train.shape, test.shape

((59400, 41), (14358, 40))

In [4]:
## import more libraries

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Split Train into train and val

In [5]:
## Split train into train & val

from sklearn.model_selection import train_test_split

train, val = train_test_split(train, train_size = 0.80, test_size= 0.20, 
                              stratify = train['status_group'], random_state = 14)
train.shape, val.shape

((47520, 41), (11880, 41))

# Data Wrangling

In [6]:
## I had used my own strategy the past two days and today I want to use
## the wrangle methods used in class

def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # Encode latitude
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # replace 0's
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
                       'gps_height', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
            
    # Drop duplicate columns
    duplicates = ['quantity_group', 'extraction_type_class', 
                  'extraction_type_group', 'scheme_management', 
                  'payment_type', 'quality_group', 'waterpoint_type_group']
    X = X.drop(columns=duplicates)
    
    # Drop recorded_by (never varies) and id (always varies, random)
    unusable_variance = ['recorded_by', 'id']
    X = X.drop(columns=unusable_variance)
    
    # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract components from date_recorded, then drop the original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    # Engineer feature: how many years from construction_year to date_recorded
    X['years'] = X['year_recorded'] - X['construction_year']    
   
    # return the wrangled dataframe
    return X

train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

# Arrange data into features matrix and target vector

In [7]:
## identify target vector
target = 'status_group'

In [8]:
## Arrange data into X features matrix and y target vector 
X_train = train.drop(labels = 'status_group', axis = 1)
y_train = train['status_group']
X_val = val.drop(labels = 'status_group', axis = 1)
y_val = val['status_group']
X_test = test

# Create and use a pipeline

In [13]:
## change the pipline
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline


pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='most_frequent'),  
    RandomForestClassifier(n_estimators=72, random_state=14, n_jobs=-1)
)

## Fit on train
pipeline.fit(X_train, y_train)

## Score on val
print('Validation Accuracy', pipeline.score(X_val, y_val))

## Predict on test
y_pred = pipeline.predict(X_test)

Validation Accuracy 0.8073232323232323


# Randomized Cross Validation

In [14]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

In [19]:
from sklearn.metrics import accuracy_score, make_scorer
accuracy_scorer = make_scorer(accuracy_score)

In [22]:
param_distributions = {
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'], 
    'randomforestclassifier__n_estimators': randint(50, 500), 
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None], 
    'randomforestclassifier__max_features': uniform(0, 1), 
}


search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=10, 
    cv=3, 
    scoring=accuracy_scorer, 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed:  4.5min remaining:  2.6min
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:  5.1min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed:  5.9min remaining:   39.4s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  6.0min finished


In [23]:
pipeline = search.best_estimator_

In [25]:
y_pred = pipeline.predict(X_test)

In [26]:
y_pred.shape

(14358,)

In [27]:
df = pd.read_csv('qureshi_submission2.csv')
df.head()

,id,Status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [29]:
(df['Status_group'] == y_pred).value_counts()

True     13343
False     1015
Name: Status_group, dtype: int64

In [30]:
df['Status_group'] = y_pred
qureshi_submission = df
qureshi_submission.to_csv('qureshi_submission3.csv', index = False)